Fraction of Missing Information

This notebook computes FMI diagnostics for renamed survey files.
It builds on:
- Config loader (`config.json`) for BASE_PATH, INTERIM_DIR, PROCESSED_DIR, LOG_DIR
- Inventory (`inventory.json`) for locating renamed survey files
- Consistency profile (`consistency_profile.csv`) for tagging variables

Outputs:
- Per‑month FMI reports in PROCESSED_DIR/FMI_Reports/<year>/
- Consolidated FMI summary in NEW Variable Consistency Check/fmi_profile.csv
- Log file in LOG_DIR/fmi_report.log

Key improvements over old FMI automation:
- No hardcoded paths (uses config loader from 00)
- Canonical variable alignment
- Integration with consistency profile
- Modular missingness detection
- Unified outputs and logging


In [1]:

import json
from pathlib import Path
import os
import pandas as pd
import numpy as np
from datetime import datetime

# --- Load config ---
with open(Path("./data/interim/config.json")) as f:
    cfg = json.load(f)

BASE_PATH = Path(cfg["BASE_PATH"])
INTERIM_DIR = Path(cfg["INTERIM_DIR"])
PROCESSED_DIR = Path(cfg["PROCESSED_DIR"])
LOG_DIR = Path(cfg["LOG_DIR"])
MONTH_ORDER = cfg["MONTH_ORDER"]

# --- Load inventory ---
with open(Path(INTERIM_DIR) / "inventory.json") as f:
    inventory = json.load(f)

# --- Paths ---
RENAMED_ROOT = BASE_PATH / "NEW Renamed Fully Decoded Surveys"
CONSISTENCY_ROOT = BASE_PATH / "NEW Variable Consistency Check"
OUTPUT_PATH = CONSISTENCY_ROOT / "FMI_Reports"
os.makedirs(OUTPUT_PATH, exist_ok=True)

# --- Load consistency profile ---
consistency_df = pd.read_csv(CONSISTENCY_ROOT / "consistency_profile.csv")
consistency_tags = dict(zip(consistency_df["Variable"], consistency_df["ConsistencyTag"]))


In [2]:
TEXT_MISSING = {"", " ", "NA", "N/A", "NaN", "nan", ".", "-", "_"}
NUMERIC_SENTINELS = {9, 99, 999, 9999, -9, -99, -999, -9999}

def build_missing_mask(series: pd.Series,
                       include_numeric_sentinels: bool = True) -> pd.Series:
    s = series.astype(str).str.strip()
    mask = series.isna() | (s == "") | s.isin(TEXT_MISSING)
    if include_numeric_sentinels:
        s_num = pd.to_numeric(series, errors="coerce")
        mask |= s_num.isin(NUMERIC_SENTINELS)
    return mask


#### FMI Scanner per file

In [3]:
def fmi_scan_csv(file_path: Path, year: str, month: str) -> pd.DataFrame:
    df = pd.read_csv(file_path, low_memory=False)
    rows = []
    for col in df.columns:
        miss_mask = build_missing_mask(df[col])
        missing = int(miss_mask.sum())
        total = int(len(df[col]))
        fmi = (missing / total) if total > 0 else 0.0

        # Severity flags
        if fmi < 0.05:
            flag, rec = "Low", "Keep"
        elif fmi < 0.20:
            flag, rec = "Moderate", "Consider imputation"
        elif fmi < 0.40:
            flag, rec = "High", "Strongly consider imputation"
        else:
            flag, rec = "Critical", "Candidate to drop (validate with business logic)"

        rows.append({
            "Year": year,
            "Month": month,
            "Variable": col.strip(),
            "Missing": missing,
            "Total": total,
            "FMI": round(fmi, 6),
            "Flag": flag,
            "Recommendation": rec,
            "ConsistencyTag": consistency_tags.get(col.strip(), "unknown")
        })
    return pd.DataFrame(rows)


#### Batch FMI Report Generator

In [4]:
# Codeblock 4 — Batch FMI Runner with UTF-8 logging (fixed)

import os
import pandas as pd
from datetime import datetime

# Assume RENAMED_ROOT, OUTPUT_PATH, LOG_DIR, fmi_scan_csv are already defined earlier

log_file = LOG_DIR / f"fmi_report_{datetime.now().strftime('%Y%m%d_%H%M%S')}.log"

success_count, error_count = 0, 0
all_reports = []

with open(log_file, "w", encoding="utf-8") as log:
    log.write("STARTING FMI REPORT\n")
    log.write(f"Source: {RENAMED_ROOT}\n")
    log.write(f"Dest:   {OUTPUT_PATH}\n")
    log.write("===============================================\n\n")

    for year in os.listdir(RENAMED_ROOT):
        year_folder = RENAMED_ROOT / year
        if not year_folder.is_dir():
            continue

        for file in os.listdir(year_folder):
            if not file.endswith(".CSV"):
                continue

            month = file.split("_")[0].capitalize()
            file_path = year_folder / file

            try:
                report = fmi_scan_csv(file_path, year, month)
                out_file = OUTPUT_PATH / f"FMI_{month}_{year}.csv"
                report.to_csv(out_file, index=False)
                all_reports.append(report)

                success_count += 1
                log.write(f"[OK] {file} → {len(report)} variables\n")
            except Exception as e:
                error_count += 1
                log.write(f"[ERROR] {file} → {e}\n")

    log.write("\n===============================================\n")
    log.write(f"COMPLETED. Success: {success_count} | Errors: {error_count}\n")


#### Weighted Summary Across All Months

In [5]:
combined = pd.concat(all_reports, ignore_index=True)

FMI_summary = (
    combined.groupby(["Variable", "ConsistencyTag"])
    .agg(
        TotalMissing=("Missing", "sum"),
        TotalRows=("Total", "sum"),
        AvgFMI=("FMI", "mean"),
        MonthsObserved=("Year", "count")
    )
    .reset_index()
)

FMI_summary["OverallFMI"] = FMI_summary["TotalMissing"] / FMI_summary["TotalRows"]

def flag_and_rec(fmi):
    if fmi < 0.05:
        return "Low", "Keep"
    elif fmi < 0.20:
        return "Moderate", "Consider imputation"
    elif fmi < 0.40:
        return "High", "Strongly consider imputation"
    else:
        return "Critical", "Candidate to drop (validate with business logic)"

FMI_summary[["Flag", "Recommendation"]] = FMI_summary["OverallFMI"].apply(
    lambda x: pd.Series(flag_and_rec(x))
)

FMI_summary.to_csv(CONSISTENCY_ROOT / "fmi_profile.csv", index=False)
